In [33]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn import linear_model
from datetime import datetime
from datetime import timedelta


In [3]:
!pip install transformers
from transformers import pipeline

  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)


/Users/ashutoshsingh/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
xDf = pd.read_csv("outputxnew.csv").to_numpy()
yDf = pd.read_csv("outputynew.csv").to_numpy()

In [25]:
yDf

array([[ 0.00850391],
       [ 0.00850391],
       [ 0.00850391],
       ...,
       [-0.0027715 ],
       [-0.0027715 ],
       [-0.0027715 ]])

In [380]:
def Accuracy(actual,predicted):
    fails = 0;
    passes = 0;
    for i, j in zip(y_train,pred):
        if(i*j<0):
            fails=fails+1
        else:
            passes=passes+1
    return passes/(passes+fails)

In [28]:
mergedFile = pd.read_csv("merged_file.csv")

In [29]:
mergedFile.head()

tweet_id           writer   post_date   
0  550441509175443456  VisualStockRSRC  2015-01-01  \
1  550441672312512512      KeralaGuy77  2015-01-01   
2  550441732014223360      DozenStocks  2015-01-01   
3  550442977802207232     ShowDreamCar  2015-01-01   
4  550443807834402816     i_Know_First  2015-01-01   

                                                body  comment_num   
0  lx21 made $10,008  on $AAPL -Check it out! htt...            0  \
1  Insanity of today weirdo massive selling. $aap...            0   
2  S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...            0   
3  $GM $TSLA: Volkswagen Pushes 2014 Record Recal...            0   
4  Swing Trading: Up To 8.91% Return In 14 Days h...            0   

   retweet_num  like_num ticker_symbol  
0            0         1          AAPL  
1            0         0          AAPL  
2            0         0          AMZN  
3            0         1          TSLA  
4            0         1          AAPL

### selecting out the tweets of one week with top 200 tweets in retweeting

In [30]:
targetDate = "2018-08-30"
tragetName = "AMZN"

In [34]:
target_date = pd.to_datetime(targetDate)
one_week_before = target_date - timedelta(days=7)

In [35]:
one_week_before = one_week_before.strftime('%Y-%m-%d')

In [36]:
start_value = one_week_before
end_value = targetDate

resultPart = mergedFile[(mergedFile['post_date'] >= start_value) & (mergedFile['post_date'] <= end_value)]

In [38]:
resultPart.head()

tweet_id          writer   post_date   
3070631  1032417137166561280    tradewithjoe  2018-08-23  \
3070632  1032417179159879680  MelaynaLokosky  2018-08-23   
3070633  1032417639170285568       valuewalk  2018-08-23   
3070634  1032417887246594048       PlugInFUD  2018-08-23   
3070635  1032417956834226176      pharris667  2018-08-23   

                                                      body  comment_num   
3070631  100K OPTIONS TRADING CHALLENGE IS ON. ARE YOU ...            0  \
3070632  She could have written this open letter while ...            2   
3070633  Crispin Odey: Tesla Is “Entering The Final Sta...            0   
3070634  Mirror mirror on the wallWho's the baggiest ba...            0   
3070635  IBD Big Cap 20 With Rank August 22, 2018 1. $A...            0   

         retweet_num  like_num ticker_symbol  
3070631            0         0          AAPL  
3070632            1         7          TSLA  
3070633            0         3          TSLA  
3070634            0         3          TSLA  
3070635            0         8          AAPL

In [39]:
resultCompany = resultPart[resultPart['ticker_symbol'] == "AMZN"]
resultCompany = resultCompany.sort_values('retweet_num', ascending=False)
resultCompany = resultCompany.head(200)
resultCompany.head()

tweet_id          writer   post_date   
3091750  1034878896305569801  charliebilello  2018-08-29  \
3086869  1034474232074457089     crowdturtle  2018-08-28   
3091451  1034861214646644737      followtheh  2018-08-29   
3090840  1034832533547044864    Sandro_power  2018-08-29   
3092155  1034897704252530689    Sandro_power  2018-08-29   

                                                      body  comment_num   
3091750  2018 Returns...$FB: -1%$AAPL: +33%$AMZN: +70%$...            4  \
3086869  absolutely fantastic partnership letter from @...            7   
3091451  Aaron Rodgers' entire contract is 3.9% of what...            8   
3090840  while the stockclown #1 bearish wrong stock ex...           27   
3092155  Live stream Tonight at  9;30pm Eastern Standar...           17   

         retweet_num  like_num ticker_symbol  
3091750           88       125          AMZN  
3086869           77       321          AMZN  
3091451           59       147          AMZN  
3090840           57        71          AMZN  
3092155           55        73          AMZN

#### Creating other input params for prediction

In [40]:
tweetsOb = resultCompany['body'].values
retweetsOb = resultCompany['retweet_num'].values
commentOb = resultCompany['comment_num'].values
likeOb = resultCompany['like_num'].values

# How many days before the target Date:
daysOb = [(datetime.strptime(targetDate, "%Y-%m-%d")-datetime.strptime(date2, "%Y-%m-%d")).days for date2 in resultCompany['post_date']]



In [188]:
# Weighted Average

retSum = sum(retweetsOb)
retweetsObW = retweetsOb/retSum
comSum = sum(commentOb)
commentObW = commentOb/comSum
likeSum = sum(likeOb)
likeObW = likeOb/likeSum

weights = [x + y + z for x, y, z in zip(retweetsObW, commentObW, likeObW)]
# weights = [1 for i in range(100)]
sum(weights)

In [209]:
def weighted_avg(items, weights):
    if len(items) != len(weights):
        raise ValueError("Items and weights must have the same length.")
    total_weighted_sum = sum([item * weight for item, weight in zip(items, weights)])
    total_weight = sum(weights)
    if total_weight == 0:
        raise ValueError("Total weight cannot be zero.")
    weighted_avg = total_weighted_sum / total_weight
    return weighted_avg
SentimentWeightedAvg = weighted_avg(labels,weights)

### FinBert Model

In [384]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [385]:
from transformers import pipeline


In [386]:
nlp = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [390]:
sentiResult = nlp(tweetsOb.tolist())

In [11]:
sentiResult = [{'label': 'neutral', 'score': 0.9201053977012634},
 {'label': 'positive', 'score': 0.5393479466438293},
 {'label': 'neutral', 'score': 0.9033854603767395},
 {'label': 'neutral', 'score': 0.9045191407203674},
 {'label': 'neutral', 'score': 0.7440078854560852},
 {'label': 'positive', 'score': 0.5670746564865112},
 {'label': 'neutral', 'score': 0.6934283971786499},
 {'label': 'positive', 'score': 0.44679689407348633},
 {'label': 'neutral', 'score': 0.8157535195350647},
 {'label': 'neutral', 'score': 0.9311531782150269},
 {'label': 'neutral', 'score': 0.9164313077926636},
 {'label': 'neutral', 'score': 0.8958992958068848},]

In [21]:
labels = [0 for i in sentiResult]
i =0
for el in sentiResult:
    if el['label'] == 'positive':
        labels[i] = (el['score'])
    elif el['label'] == 'negative':
        labels[i] = -(el['score'])
    i=i+1


        

In [401]:
labels = [{"positive":0,"neutral":0,"negative":0} for i in sentiResult]
i =0
for obj in sentiResult:
    (labels[i])[obj['label']] = obj['score']
    i=i+1

In [403]:
labels[:5]

[{'positive': 0, 'neutral': 0.9201053977012634, 'negative': 0},
 {'positive': 0.5393479466438293, 'neutral': 0, 'negative': 0},
 {'positive': 0, 'neutral': 0.9033854603767395, 'negative': 0},
 {'positive': 0, 'neutral': 0.9045191407203674, 'negative': 0},
 {'positive': 0, 'neutral': 0.7440078854560852, 'negative': 0}]